In [ ]:
import pandas as pd
import ast
import copy
import gensim
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split


In [ ]:

df_clean = pd.read_csv("../df_clean-2.csv").drop(columns="index")

df_data = df_clean[df_clean.Target != "UNKNOWN"]
df_pred = df_clean[df_clean.Target == "UNKNOWN"]
df_data = df_data.reset_index().drop(columns=["index"])
df_pred = df_pred.reset_index().drop(columns=["index"])


map = {
"Program":        0,
"Display":        1,
"BTB":            2,
"Search":         3,
"Holiday":        4,
"BTS":            5,
"Email":          6,
"Digital":        7,
"Trad_media":     8
}

y_df = df_data["Target"].map(map)
X_df = pd.DataFrame({"Name": [ast.literal_eval(i) for i in df_data["Name"].values.tolist()], "Descr": [ast.literal_eval(i) for i in df_data["Descr"].values.tolist()]}) 

concat = []
for j, i in zip(X_df ["Name"].values, X_df ["Descr"].values):
  new = copy.deepcopy(j)
  for k in i:
    new.append(copy.deepcopy(k))
  concat.append(new)
X_df ["Concat"] = concat

X = X_df["Concat"]
y = y_df.values

In [ ]:
class model():
    def __init__(self, embeding_vector_size = 100, skipg_window = 3, min_count = 2, use_sg = 1):
        
        # entry vectors
        
        self.X = "nan"
        self.y = "nan"
        
        # last prediction
        
        self.y_pred = "nan"
        
        # expanded categories vectors 
        
        self.X_sub = "nan"
        self.y_sub = "nan"
        
        # last embeded sentences vector
        
        self.X_vect_avg = []
        
        # created subcategories visualization
        
        self.subcategories_df = "nan"
        
        # number of clusters to generate per class "K"
        
        self.n_clusters = "nan"
        
        # models 
        
        self.w2v_model = "nan"
        self.kmeanModel = "nan"
        self.svc = "nan"
        
        # w2v hyperparameters
        
        self.embeding_vector_size = embeding_vector_size
        self.skipg_window = skipg_window
        self.min_count = min_count
        self.use_sg = use_sg
        
    
    def __clusterize(self, classes_df, labels):
        new_classes_dic = {}
        for i in classes_df:
            class_labels = labels[i][self.n_clusters[i]]
            new_class_labels = []
            for j in class_labels:
                new_class_labels.append(i + " " + str(j))
            new_classes_dic[i] = pd.DataFrame({"Target": new_class_labels, "sentence": classes_df[i]["sentence"].values.tolist()})
        new_classes_dic = pd.concat(new_classes_dic.values(), ignore_index=True)
        
        self.new_map = {np.unique(new_classes_dic["Target"].values)[i]:i  for i in range(len(np.unique(new_classes_dic["Target"].values)))}
        self.y_sub = new_classes_dic["Target"].map(self.new_map)
        self.X_sub = new_classes_dic["sentence"].values.tolist()
        self.subcategories_df = pd.DataFrame({"Target key": self.y_sub,"Target": new_classes_dic["Target"], "sentence": self.X_sub})
        

    
    def fit(self, X, y, n_clusters):
        
        self.X = X
        self.y = y
        self.n_clusters = n_clusters
        
        # training w2v
        
        self.w2v_model = gensim.models.Word2Vec(sentences = self.X,
                                   vector_size=self.embeding_vector_size,
                                   window=self.skipg_window,
                                   min_count=self.min_count,
                                   sg=self.use_sg)
        
        # arranging sentences with embeded words
        
        words = set(self.w2v_model.wv.index_to_key )
        X_vect = np.array([np.array([self.w2v_model.wv[i] for i in ls if i in words]) for ls in X])

        # averaging words into sentence

        
        for v in X_vect:
            if v.size:
                self.X_vect_avg.append(v.mean(axis=0))
            else:
                self.X_vect_avg.append(np.zeros(vector_size, dtype=float))
                
        # dividing the different categories to prepare them for clustering
                
        df = pd.DataFrame({"Target": self.y, "sentence": self.X_vect_avg})
 
        classes = {i: df[df.Target == map[i]] for i in map}
        for i in classes:
            classes[i] = classes[i].reset_index().drop(columns=["index"])
        
        # clustering from k = 1 to k = 12 to match the given map
        
        labels = {}

        for i in classes:
            i_labels = {}
            K = range(1,12)
            for k in K:
                self.kmeanModel = KMeans(n_clusters=k, algorithm="elkan").fit(classes[i]['sentence'].values.tolist())
                i_labels[k] = self.kmeanModel.labels_
            labels[i] = i_labels
            
        # arranging and selecting clustering results
        
        self.__clusterize(classes, labels)
        
        # training SVC
        
        self.svc = SVC(kernel = "linear").fit(self.X_sub, self.y_sub)
        
    def predict(self, X):
        
        # arranging sentences with embeded words
        
        words = set(self.w2v_model.wv.index_to_key)
        X_vect = np.array([np.array([self.w2v_model.wv[i] for i in ls if i in words]) for ls in X])

        # averaging words into sentence

        self.X_vect_avg = []
        
        for v in X_vect:
            if v.size:
                self.X_vect_avg.append(v.mean(axis=0))
            else:
                self.X_vect_avg.append(np.zeros(self.embeding_vector_size, dtype=float))

                
        self.svc.prediction = self.svc.predict(self.X_vect_avg)
        
        return self.svc.prediction

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

n_clusters = {
"Program":        10,
"Display":        5,
"BTB":            3,
"Search":         4,
"Holiday":        5,
"BTS":            5,
"Email":          2,
"Digital":        1,
"Trad_media":     1
}

mod = model()
mod.fit(X_train ,y_train, n_clusters)
y_pred = mod.predict(X_test)


In [ ]:
def map_test(y_pred, original_map):
        n_map = {}
        for i in mod.new_map:
            for j in original_map:
                if(j == i[:len(j)]):
                    n_map[mod.new_map[i]] = original_map[j]
                    break
        return pd.Series(y_pred).map(n_map).values

In [ ]:
accuracy_score(y_test, map_test(y_pred, map))